In [215]:
import pandas as pd
import numpy as np
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [208]:
dataframe = pd.read_csv("GSE113865_series_matrix.txt", delimiter='\t')
#print(dataframe.head())
dataset=dataframe.values
#print(dataset)
x1=dataset[:,1:7].astype(float)
x1 = x1.T
target=np.array([0,1,0,1,0,1])
#y=np.tile(target,(47320,1))
y=np.array([0,1,0,1,0,1])
x1.shape
y.shape
x1.max()
x1.min()

5.975281

In [209]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x1)
x = x.astype('float32')
y = y.astype('float32')

#print(f'X shape: {x.shape}, X dtype: {x.dtype}')
#print(f'y shape: {y.shape}, y dtype: {y.dtype}')

In [236]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42)

In [252]:
#baseline model creation


model = Sequential()
model.add(Dense(64, input_dim=x.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1 if len(np.unique(y)) == 2 else len(np.unique(y)), activation='sigmoid' if len(np.unique(y)) == 2 else 'softmax'))

    
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(x_train, y_train, 
                    epochs=16, 
                    batch_size=50, 
                    validation_data=(x_test, y_test))

Model: "sequential_80"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_234 (Dense)                    │ (None, 64)                  │       3,028,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_108 (Dropout)                │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_235 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_109 (Dropout)                │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_236 (Dense)                    │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,030,657 (11.56 MB)

 Trainable params: 3,030,657 (11.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2000 - loss: 2.8602 - val_accuracy: 1.0000 - val_loss: 1.6031
Epoch 2/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.6000 - loss: 3.9321 - val_accuracy: 1.0000 - val_loss: 1.5285
Epoch 3/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2000 - loss: 12.4405 - val_accuracy: 1.0000 - val_loss: 1.4837
Epoch 4/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6000 - loss: 9.9990 - val_accuracy: 1.0000 - val_loss: 1.4552
Epoch 5/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6000 - loss: 4.0428 - val_accuracy: 1.0000 - val_loss: 1.5965
Epoch 6/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.4000 - loss: 15.7226 - val_accuracy: 0.0000e+00 - val_loss: 7.9525
Epoch 7/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.4000 - loss: 23.9733 - val_accuracy: 0.0000e+00 - val_loss: 14.3195
Epoch 8/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6000 - loss: 5.4063 - val_accuracy: 0.0000e+00 - val

In [253]:
score = model.evaluate(x, y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.6424152851104736
Test accuracy: 0.8333333134651184
